In [1]:
import pandas as pd
from rdkit import Chem
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import keras
from tabulate import tabulate
from sklearn.cluster import KMeans
from rdkit.Chem import MACCSkeys,MolFromSmiles
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error

In [2]:
#Converting the csv file into dataframe
df = pd.read_csv('pilot_library.csv')
#display(df)

In [3]:
# Add a column for RDKit molecule objects
df['Molecule'] = df['smiles'].apply(Chem.MolFromSmiles)
df['Fingerprint'] = df['Molecule'].apply(lambda x: MACCSkeys.GenMACCSKeys(x))
fingerprints = np.array(df['Fingerprint'].tolist())

In [4]:
"""
#Calculate the Tanimoto distance to get the similarity matrix
def calculate_tanimoto_similarity(fp1, fp2):
    intersection = np.sum(np.logical_and(fp1, fp2))
    union = np.sum(np.logical_or(fp1, fp2))
    if union == 0:
        return 0.0
    else:
        return intersection / union

fingerprints = np.array(df['Fingerprint'].tolist())
similarity_matrix = np.zeros((len(fingerprints), len(fingerprints)))

for i in range(len(fingerprints)):
    for j in range(i, len(fingerprints)):
        tanimoto_similarity = calculate_tanimoto_similarity(fingerprints[i], fingerprints[j])
        similarity_matrix[i, j] = tanimoto_similarity
        similarity_matrix[j, i] = tanimoto_similarity
"""

"\n#Calculate the Tanimoto distance to get the similarity matrix\ndef calculate_tanimoto_similarity(fp1, fp2):\n    intersection = np.sum(np.logical_and(fp1, fp2))\n    union = np.sum(np.logical_or(fp1, fp2))\n    if union == 0:\n        return 0.0\n    else:\n        return intersection / union\n\nfingerprints = np.array(df['Fingerprint'].tolist())\nsimilarity_matrix = np.zeros((len(fingerprints), len(fingerprints)))\n\nfor i in range(len(fingerprints)):\n    for j in range(i, len(fingerprints)):\n        tanimoto_similarity = calculate_tanimoto_similarity(fingerprints[i], fingerprints[j])\n        similarity_matrix[i, j] = tanimoto_similarity\n        similarity_matrix[j, i] = tanimoto_similarity\n"

In [5]:
"""
#Create a 2D array representing the first 2 principle components
pca = PCA(n_components=3)
dissimilarity_matrix = 1 - similarity_matrix
points_2d = pca.fit_transform(similarity_matrix)
pca_df = pd.DataFrame(
    data=points_2d, 
    columns=['Principal Component 1','Principal Component 2','Principal Component 3'])
display(pca_df)
"""

"\n#Create a 2D array representing the first 2 principle components\npca = PCA(n_components=3)\ndissimilarity_matrix = 1 - similarity_matrix\npoints_2d = pca.fit_transform(similarity_matrix)\npca_df = pd.DataFrame(\n    data=points_2d, \n    columns=['Principal Component 1','Principal Component 2','Principal Component 3'])\ndisplay(pca_df)\n"

In [6]:
"""num_components_to_display = 3

# Create a DataFrame with principal components and explained variance ratios
components_df = pd.DataFrame({
    'Principal Component': [f'PC{i+1}' for i in range(num_components_to_display)],
    'Explained Variance Ratio': pca.explained_variance_ratio_[:num_components_to_display]
})

# Display in tabular format
table = tabulate(components_df, headers='keys', tablefmt='pretty', showindex=False)
print(table)

"""

"num_components_to_display = 3\n\n# Create a DataFrame with principal components and explained variance ratios\ncomponents_df = pd.DataFrame({\n    'Principal Component': [f'PC{i+1}' for i in range(num_components_to_display)],\n    'Explained Variance Ratio': pca.explained_variance_ratio_[:num_components_to_display]\n})\n\n# Display in tabular format\ntable = tabulate(components_df, headers='keys', tablefmt='pretty', showindex=False)\nprint(table)\n\n"

In [7]:
#K-mean Clustering
num_clusters = 2 
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['Cluster'] = kmeans.fit_predict(fingerprints)

"""
#Plot the 2D array and coloring by cluster
plt.figure(figsize=(10, 8))
sns.scatterplot(x=points_2d[:, 0], y=points_2d[:, 1], hue=df['Cluster'], palette='viridis', alpha=0.7)
plt.title('Molecules in 2D Space with Kmeans Clustering')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster')
plt.show()"""

"\n#Plot the 2D array and coloring by cluster\nplt.figure(figsize=(10, 8))\nsns.scatterplot(x=points_2d[:, 0], y=points_2d[:, 1], hue=df['Cluster'], palette='viridis', alpha=0.7)\nplt.title('Molecules in 2D Space with Kmeans Clustering')\nplt.xlabel('Principal Component 1')\nplt.ylabel('Principal Component 2')\nplt.legend(title='Cluster')\nplt.show()"

In [8]:
#K-mean Clustering

num_clusters = 2
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['Cluster'] = kmeans.fit_predict(fingerprints)

"""
# 3D scatter plot with cluster colors
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(111, projection='3d')

# Use the first three principal components for x, y, and z axes
ax.scatter(points_2d[:, 0], points_2d[:, 1], points_2d[:, 2], c=df['Cluster'], cmap='viridis', alpha=0.7)

ax.set_title('Molecules in 3D Space with K-Means Clustering')
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')

plt.show()
"""

"\n# 3D scatter plot with cluster colors\nfig = plt.figure(figsize=(20, 20))\nax = fig.add_subplot(111, projection='3d')\n\n# Use the first three principal components for x, y, and z axes\nax.scatter(points_2d[:, 0], points_2d[:, 1], points_2d[:, 2], c=df['Cluster'], cmap='viridis', alpha=0.7)\n\nax.set_title('Molecules in 3D Space with K-Means Clustering')\nax.set_xlabel('Principal Component 1')\nax.set_ylabel('Principal Component 2')\nax.set_zlabel('Principal Component 3')\n\nplt.show()\n"

In [9]:
#Select 30 random molecules from the dataframe for the 2 clusters  
group_1 = df[df['Cluster'] == 0]
group_2 = df[df['Cluster'] == 1]

print(len(group_1))

random_1 = group_1.sample(n=30) 
random_2 = group_2.sample(n=30) 

Cluster_1 = pd.DataFrame(random_1)
Cluster_2 = pd.DataFrame(random_2)

#Save the chosen molecules into a csv file
csv_1 = "Cluster_1.csv"
csv_2 = "Cluster_2.csv"

Cluster_1.to_csv(csv_1, index=False)
Cluster_2.to_csv(csv_2, index=False)

#display(group_1)


1068


# Regression

In [10]:
#Adding the 3 known inhibitors into our dataset
acetylcysteine_smiles = "CC(=O)NC(CS)C(=O)O"
clavulanic_acid = "C1C2N(C1=O)C(C(=CCO)O2)C(=O)O"
homovanillic_acid = "COC1=C(C=CC(=C1)CC(=O)O)O"
c1 = pd.read_csv('Scores1.csv')
c2 = pd.read_csv('Scores1.csv')

def merge(c1,group_1):
    merged_df = pd.merge(c1, df, left_on='Smile', right_on='smiles', how='left')
    c1['fingerprint'] = merged_df['Fingerprint']

    existing_names = set(c1['Name'])
    existing_smiles = set(c1['Smile'])

    filtered_group_1 = group_1[~(group_1['eos'].isin(existing_names) | group_1['smiles'].isin(existing_smiles))]
    g_1 = filtered_group_1[['eos','smiles','Fingerprint']]

    return c1, g_1

c1 , g_1 = merge(c1,group_1)
c2 , g_2 = merge(c2,group_2)
display(len(c1))

def add_compounds_to_dataframe(df, compounds, smiles_list):
    rows = []
    for compound, smiles in zip(compounds, smiles_list):
        mol = MolFromSmiles(smiles)
        fp = MACCSkeys.GenMACCSKeys(mol)
        #print(len(fp))
        new_row = {'eos': compound, 'smiles': smiles, 'Fingerprint': fp}
        rows.append(new_row)
    new_rows_df = pd.DataFrame(rows)
    df = pd.concat([df, new_rows_df], ignore_index=True)
    return df

g_1 = add_compounds_to_dataframe(g_1, ['Acetylcysteine', 'Clavulanic Acid', 'Homovanillic Acid'],[acetylcysteine_smiles, clavulanic_acid, homovanillic_acid])
g_2 = add_compounds_to_dataframe(g_2, ['Acetylcysteine', 'Clavulanic Acid', 'Homovanillic Acid'],[acetylcysteine_smiles, clavulanic_acid, homovanillic_acid])

66

In [11]:
"""
#Filter the dataframe using the sum of the fingerprints and taking into account only the one inferieur to 50
def calculate_row_fingerprint_sum(row):
    fingerprint_sum = sum(row['Fingerprint'])
    return fingerprint_sum

g_1['Fingerprint_Sum']  = g_1.apply(calculate_row_fingerprint_sum, axis=1)
g_2['Fingerprint_Sum']  = g_2.apply(calculate_row_fingerprint_sum, axis=1)

g_1 = g_1[g_1['Fingerprint_Sum'] <= 50].copy()
g_2 = g_2[g_2['Fingerprint_Sum'] <= 50].copy()
display(len(g_1))
display(len(g_2))
#display(g_2)
"""

"\n#Filter the dataframe using the sum of the fingerprints and taking into account only the one inferieur to 50\ndef calculate_row_fingerprint_sum(row):\n    fingerprint_sum = sum(row['Fingerprint'])\n    return fingerprint_sum\n\ng_1['Fingerprint_Sum']  = g_1.apply(calculate_row_fingerprint_sum, axis=1)\ng_2['Fingerprint_Sum']  = g_2.apply(calculate_row_fingerprint_sum, axis=1)\n\ng_1 = g_1[g_1['Fingerprint_Sum'] <= 50].copy()\ng_2 = g_2[g_2['Fingerprint_Sum'] <= 50].copy()\ndisplay(len(g_1))\ndisplay(len(g_2))\n#display(g_2)\n"

# Random Forest

In [12]:
def prediction(g_1,c1):
    binary_fingerprints = [list(fp.ToBitString()) for fp in g_1['Fingerprint']]
    fingerprint_features = pd.DataFrame(binary_fingerprints, columns=[f'bit_{i}' for i in range(len(binary_fingerprints[0]))])

    bb = [list(fp.ToBitString()) for fp in c1['fingerprint']]
    f = pd.DataFrame(bb, columns=[f'bit_{i}' for i in range(len(bb[0]))])

    X = fingerprint_features
    X_train = f
    y_train = c1['Conf']
    #print(y_train)

    rf_model = RandomForestRegressor()
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X)

    g_1['Conf'] = y_pred
    return g_1

pred_1 = prediction(g_1, c1)
pred_2 = prediction(g_2, c2)

#display(pred_1)
# Evaluation
"""mse = mean_squared_error(y, y_pred)
print("Mean Squared Error:", mse)"""


'mse = mean_squared_error(y, y_pred)\nprint("Mean Squared Error:", mse)'

# Neual Network

In [13]:
def prediction_neural_network(g_1, c1):
    binary_fingerprints = [list(fp.ToBitString()) for fp in g_1['Fingerprint']]
    fingerprint_features = pd.DataFrame(binary_fingerprints, columns=[f'bit_{i}' for i in range(len(binary_fingerprints[0]))])

    bb = [list(fp.ToBitString()) for fp in c1['fingerprint']]
    f = pd.DataFrame(bb, columns=[f'bit_{i}' for i in range(len(bb[0]))])
    
    X = fingerprint_features
    X_train = f.astype('float32')
    y_train = c1['Conf'].astype('float32').values.reshape(-1, 1)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    # Define a neural network model
    model = Sequential([Dense(64, activation='relu', input_shape=(X_train.shape[1],)),Dense(32, activation='relu'),Dense(1)])
    
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    X = scaler.transform(X) 
    y_pred = model.predict(X)

    g_1['Conf'] = y_pred.flatten()
    return g_1

# Prediction
def prediction_neural_network_stable(g_1, c1, num_iterations=50):
    all_predictions = []

    for i in range(num_iterations):
        y_pred = prediction_neural_network(g_1, c1)['Conf'].values
        all_predictions.append(y_pred)

    average_prediction = np.mean(all_predictions, axis=0)
    g_1['Conf_Average'] = average_prediction

    return g_1

pred_1 = prediction_neural_network_stable(g_1, c1)
pred_2 = prediction_neural_network_stable(g_2, c2)





124/124 [==============================] - 0s 2ms/step


In [ ]:
def target(pred):
    target = pred[pred['Conf_Average'] > -0.5]
    return target

target_1 = target(pred_1)
target_2 = target(pred_2)
display(len(target_1))
display(len(target_2))
display(target_1)

In [15]:
#Generate the target files in csv containing the molecules Confidance Average superior to -0.5
csv_1 = "Cluster_1_predictions.csv"
csv_2 = "Cluster_2_predictions.csv"

target_1.to_csv(csv_1, index=True)
target_2.to_csv(csv_2, index=True)

# Evaluating the neural network on our data

In [16]:
#Getting the predictions on just the molecules that have their DiffDock confidence alredy calculated
def neural_network(c1, confidence_column='Conf'):
    binary_fingerprints = [list(fp.ToBitString()) for fp in c1['fingerprint']]
    fingerprint_features = pd.DataFrame(binary_fingerprints, columns=[f'bit_{i}' for i in range(len(binary_fingerprints[0]))])
    
    X = fingerprint_features
    y = c1[confidence_column].astype('float32').values.reshape(-1, 1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = X_test.astype(float)
    
    model = Sequential([Dense(64, activation='relu', input_shape=(X_train.shape[1],)),Dense(32, activation='relu'),Dense(1)])
    
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

    y_pred = model.predict(X_test)
    y_pred = np.squeeze(y_pred) 

    return y_pred,X_test

# Prediction
def stable(c1, num_iterations=50):
    all_predictions = []

    for i in range(num_iterations):
        y_pred,X_test = neural_network(c1)#['Conf'].values
        all_predictions.append(y_pred)

    average_prediction = np.mean(all_predictions, axis=0)
    result_df = pd.DataFrame({'eos': c1['Name'].iloc[X_test.index].values, 
    'SMILES': c1['Smile'].iloc[X_test.index].values,
    'Conf': c1['Conf'].iloc[X_test.index].values,
    'Average': average_prediction})
    return result_df

pred_1 = stable(c1)
pred_2 = stable(c2)


1/1 [==============================] - 0s 66ms/step


In [17]:
#Calculating the MSE and MEA on the predicted values
mae1 = mean_absolute_error(pred_1['Conf'], pred_1['Average'])
mse1 = mean_squared_error(pred_1['Conf'],  pred_1['Average'])

mae2 = mean_absolute_error(pred_2['Conf'], pred_2['Average'])
mse2 = mean_squared_error(pred_2['Conf'],  pred_2['Average'])
mae1_formatted = f"{mae1:.4f}"
mse1_formatted = f"{mse1:.4f}"
mae2_formatted = f"{mae2:.4f}"
mse2_formatted = f"{mse2:.4f}"

data = [
    ["Cluster 1", mae1_formatted, mse1_formatted],
    ["Cluster 2", mae2_formatted, mse2_formatted]
]
headers = ["Cluster", "MAE", "MSE"]

# Tabulate
table = tabulate(data, headers=headers, tablefmt="grid")

print(table)

+-----------+--------+--------+
| Cluster   |    MAE |    MSE |
+===========+========+========+
| Cluster 1 | 0.3144 | 0.1278 |
+-----------+--------+--------+
| Cluster 2 | 0.3084 | 0.1138 |
+-----------+--------+--------+


# Evaluating ou model on the given molecules

In [18]:
#Testing the neural network on the dataset given to us by the Prague team. Their top 100 molecules
res = pd.read_csv('updated_repurposing_result.csv')
r = res[['proteinID','SMILES','mean']]

def get_maccs_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        return MACCSkeys.GenMACCSKeys(mol)
    else:
        return None

r['MACCS_Fingerprint'] = r['SMILES'].apply(get_maccs_fp)

r = r.dropna(subset=['MACCS_Fingerprint'])
#display(res)

[22:26:18] SMILES Parse Error: syntax error while parsing: OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1
[22:26:18] SMILES Parse Error: Failed parsing SMILES 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1' for input: 'OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12948\171775106.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r['MACCS_Fingerprint'] = r['SMILES'].apply(get_maccs_fp)


In [19]:
#Prediction on the 100 molecules
def prediction_neural(r):
    binary_fingerprints = [list(fp.ToBitString()) for fp in r['MACCS_Fingerprint']]
    fingerprint_features = pd.DataFrame(binary_fingerprints, columns=[f'bit_{i}' for i in range(len(binary_fingerprints[0]))])
    
    X = fingerprint_features
    y = r['mean'].astype('float32').values.reshape(-1, 1)


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = X_test.astype(float)
    #display(X_train.shape)
    
    model = Sequential([Dense(64, activation='relu', input_shape=(X_train.shape[1],)),Dense(32, activation='relu'),Dense(1)])
    
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

    y_pred = model.predict(X_test)
    y_pred = np.squeeze(y_pred) 

    result_df = pd.DataFrame({'proteinID': r['proteinID'].iloc[X_test.index].values, 
    'SMILES': r['SMILES'].iloc[X_test.index].values,
    'Mean':r['mean'].iloc[X_test.index].values,
    'Average': y_pred.flatten()})
    return result_df


prediction = prediction_neural(r)
prediction['M'] = prediction['Mean'] - prediction['Average']
#display(prediction)

1/1 [==============================] - 0s 65ms/step


In [20]:
#Calculating the MSE and MEA on the predicted values
mae = mean_absolute_error(prediction['Mean'], prediction['Average'])
mse = mean_squared_error(prediction['Mean'],  prediction['Average'])
print('Mean Absolute Error (MAE):', mae)
print('The MSE',mse)

Mean Absolute Error (MAE): 3.0114934555604496
The MSE 14.339772077774379
